# ECG Processing Example

<div class="alert alert-block alert-info">
This example notebook illustrates how to import and process ECG data per subject and save intermediate processing results so that further analysis can be performed (e.g., in <code>ECG_Analysis_Example.ipynb</code> or in <code>MIST_TSST_Example.ipynb</code>).
</div>

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp
import biopsykit.signals.ecg as ecg
from biopsykit.signals.ecg import EcgProcessor
import biopsykit.utils.array_handling as array_handling

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
plt.rcParams['figure.figsize'] = (10,5)
sns.set(style='ticks')

## Example 1: 1 Dataset, 1 Phase

This example assumes that we have one dataset with only one phase, i.e. the dataset does not need to be split into multiple parts internally.

In [ ]:
ecg_data, sampling_rate = bp.example_data.get_ecg_example()
ep = EcgProcessor(df=ecg_data, sampling_rate=sampling_rate)

### Process ECG Signal
Calling `ep.ecg_process()` will perform R peak detection and perform outlier correcrtion with the default outlier correction pipeline

In [ ]:
ep.ecg_process()

#### Optional: Use other outlier correction parameters
Calling `ep.outlier_params_default()` will list available outlier correction parameters and their default parameter. See the doumentation of `ep.outlier_corrections()` for further information.


In [ ]:
# List available outlier correction parameters and their default parameter. See the doumentation of 'EcgProcessor.outlier_corrections()' for further information.
# print(ep.outlier_params_default())
# ep.ecg_process(outlier_correction=['statistical_rr', 'statistical_rr_diff', 'physiological'], outlier_params={'statistical_rr': 2.576, 'statistical_rr_diff': 1.96, 'physiological': (50, 180)})

#### ECG and Heart Rate Result

In [ ]:
print(ep.ecg_result)

In [ ]:
# Get heart rate and print resulting heart rate 
hr_data = ep.heart_rate['Data']
hr_data.head()

In [ ]:
# Plot an overview of the acquired data
fig, axs = ecg.plotting.ecg_plot(ep, key='Data', figsize=(10,5))

### Compute HRV
Heart rate variability (HRV) is computed over the complete interval. If you want to compute HRV over different subintervals, you need to split the data first.

In [ ]:
ep.hrv_process(ep, 'Data', index='Vp01', index_name="subject_id")

#### Plot HRV Overview

In [ ]:
fig, axs = ecg.plotting.hrv_plot(ep, 'Data', figsize=(10,5))

## Example 2: 1 Dataset, Multiple Phases

This example illustrates the pipeline for one single dataset which contains data from multiple phases.

In [ ]:
ecg_data, sampling_rate = bp.example_data.get_ecg_example()

For this example, we use the example ECG Data and assume we want to split it into 3 phases (names: Preparation, Stress, Recovery) of 3 minutes

In [ ]:
#provide list of edge times (start times of the phases and the total end time)
time_intervals = pd.Series(["12:32", "12:35", "12:38", "12:41"], index=["Preparation", "Stress", "Recovery", "End"])
# alternatively: provide dict with start-end times and names per phase
#time_intervals = {"Preparation": ("12:32", "12:35"), "Stress": ("12:35", "12:38"), "Recovery": ("12:38", "12:41")}

### Process all Phases

In [ ]:
ep = EcgProcessor(df=ecg_data, sampling_rate=sampling_rate, time_intervals=time_intervals)
ep.ecg_process()

#### Compute HRV parameters for each Phase
Using List Comprehension (calling `EcgProcessor.hrv_process()` for each phase) and concat the results into one dataframe using `pd.concat()`

In [ ]:
hrv_result = pd.concat([ep.hrv_process(ep, key=key, index=key) for key in ep.phases])
# alternatively: call EcgProcessor.hrv_batch_process()
# hrv_result = ep.hrv_batch_process()

In [ ]:
hrv_result

### Plot one Phase

In [ ]:
fig, axs = ecg.plotting.ecg_plot(ep, key='Stress', figsize=(10,5))

In [ ]:
fig, axs = ecg.plotting.hrv_plot(ep, key='Stress', figsize=(10,5))

## Example 3: Multiple Subjects, Multiple Phases per Recording (Example Processing)

In [ ]:
ecg_data, sampling_rate = bp.example_data.get_ecg_example()
ecg_data_02, sampling_rate = bp.example_data.get_ecg_example_02()

For this example, we use two ECG example datasets, where the last phase ("Recovery") differs in length

In [ ]:
subject_dict = {
    'Vp01': {
        'Data': ecg_data, 
        'Time': pd.Series(["12:32", "12:35", "12:38", "12:41"], index=["Preparation", "Stress", "Recovery", "End"])
    }, 
    'Vp02': {
        'Data': ecg_data_02,
        # The last phase of Vp02 has a length of only 1 minute to demonstrate the functionality of cutting to equal length
        'Time': pd.Series(["12:55", "12:58", "13:01", "13:02"], index=["Preparation", "Stress", "Recovery", "End"])
    }
}

In [ ]:
from tqdm.notebook import tqdm # to visualize for-loop progress

# dicionaries to store processing results
dict_hr_subjects = {}
dict_hrv_result = {}

for subject_id, data_dict in tqdm(subject_dict.items(), desc="Subjects"):
    
    ep = EcgProcessor(df=data_dict['Data'], time_intervals=data_dict['Time'], sampling_rate=sampling_rate)
    ep.ecg_process(title=subject_id)
    
    dict_hr_subjects[subject_id] = ep.heart_rate
    
    hrv_result = ep.hrv_batch_process()
    dict_hrv_result[subject_id] = hrv_result

### For the further steps of the Processing Pipeline, please refer to **Example 4**

## Example 4: Multiple Subjects, Multiple Phases per Recording (Batch Processing)

This example illustrates how to set up a proper data analysis pipeline to process multiple subjects in a loop. It consists of the following steps:
1. **Get Time Information**: Load Excel sheet with time information (to process multiple subjects in a loop)
1. **Query Data**: Iterate through a folder and search for all files with a specific file pattern  
    *optional*: Iterate through a folder that contains *subfolders* for each subjects where data is stored  
    *optional*: Extract Subject ID either from data filename or from folder name
    1. Load ECG Dataset, split into phases based on time information from Excel sheet
    1. Perform ECG processing with outlier correction
    1. Compute HR, HRV, ... parameters per subject and phase
1. **Store and Export Intermediate Results**: Store Heart Rate processing results in *HR subject dict* (nested dict containing results of all subjects) and export Heart Rate processing results per subject (as Excel files)
1. **Interpolate and Resample Data**: all Heart Rate data is resampled to 1 Hz  
    *optional*: Cut phase of each subject to the shortest duration  
1. **Normalize Data** (*optional*): Normalize Heart Rate data with respect to the mean Heart Rate of the first phase (*Baseline*)
1. **Rearrange Data** Data from *HR subject dict* is rearranged so that data of subjects is in one dataframe per phase for all subjects (denoted as *Phase dict*)
1. **Export Result Parameter**: Export *Phase dict* as Excel file and Export ECG

### Get Time Information

In [ ]:
# load time log file
timelog_path = Path("../example_data/ecg_time_log.xlsx")

df_time_log = bp.io.load_time_log(timelog_path, index_cols=["ID", "Condition"])
df_time_log

### Query Data

In [ ]:
ecg_base_path = Path("../example_data/ecg")
file_list = list(sorted(ecg_base_path.glob("*.bin")))
print(file_list)

In [ ]:
from tqdm.notebook import tqdm # to visualize for-loop progress

# dicionaries to store processing results
dict_hr_subjects = {}
dict_hrv_result = {}

# for loop to iterate though subjects
for file_path in tqdm(file_list, desc="Subjects"):
    # optional: extract Subject ID from file name; multiple ways, e.g. using regex or by splitting the filename string
    subject_id = file_path.name.split('.')[0].split('_')[-1]
    
    # optional: if data is stored in subfolders: additional .glob() on file_path, get subject_id from folder name
    # ecg_files = list(sorted(file_path.glob("*")))
    # subject_id = file_path.name
    
    # check if folder contains data
    # if len(ecg_files) == 0:
        # print("No data for subject {}.".format(subject_id))
    
    # the next step depends on the file structure:
    # if you only have one recording per subject: load this file
    # df, fs = bp.io.load_dataset_nilspod(ecg_files[0])
    # if you have more than one recording per subject: loop through them, load the files and e.g. put them into a dictionary
    
    # load dataset
    df, fs = bp.io.nilspod.load_dataset_nilspod(file_path)
    
    ep = EcgProcessor(df=df, time_intervals=df_time_log.loc[subject_id], sampling_rate=256.0)
    ep.ecg_process(title=subject_id)
    
    # save ecg processing result into HR subject dict
    dict_hr_subjects[subject_id] = ep.heart_rate
    
    # compute heart rate variability for each phase separately using EcgProcessor.hrv_patch_process()
    hrv_result = ep.hrv_batch_process()
    # store HRV results in dictionary
    dict_hrv_result[subject_id] = hrv_result

### Store and Export Intermediate Results

In [ ]:
# create result directories
export_path = Path('./results')
export_path.mkdir(parents=True, exist_ok=True)
ecg_export_path = export_path.joinpath("ecg")
ecg_export_path.mkdir(parents=True, exist_ok=True)

# save HR data for separately for each subject
for subject_id, dict_subject in dict_hr_subjects.items():
    bp.io.ecg.write_hr_subject_dict(dict_subject, ecg_export_path.joinpath("hr_result_{}.xlsx".format(subject_id)))
    
# alternatively: save it directly after processing the ECG signal in the Section "Query Data".
# Then you can also directly pass the 'EcgProcessor' instance of the subject to the function:
# bp.signals.ecg.io.write_hr_subject_dict(ep, export_path.joinpath("hr_result_{}.xlsx".format(subject_id)))

### Interpolate and Resample Data

Interpolate (resample) all heart rate values to a sampling rate of 1 Hz (`utils.interpolate_dict_sec()`)  
*optional*: If phases are not the same length per subject, they might need to be cut to same length, then use `utils.interpolate_and_cut()` instead.

In [ ]:
# dict_result = utils.interpolate_dict_sec(dict_hr_subjects)
# optional: interpolate and cut in one step
dict_result = bp.utils.array_handling.interpolate_and_cut(dict_hr_subjects)

### Normalize Data (optional)

Normalize Heart Rate data with respect to the mean Heart Rate of the first phase (*Baseline*)

In [ ]:
dict_result_norm = bp.signals.ecg.normalize_heart_rate(dict_result, normalize_to='Baseline')

### Rearrange Data

Rearrange the nested dictionary (*HR subject dict*) into a *Phase dict*, i.e. a dictionary with combined heart rate values of all subjects per phase

In [ ]:
dict_phase = bp.utils.data_processing.concat_phase_dict(dict_result)
dict_phase_norm = bp.utils.data_processing.concat_phase_dict(dict_result_norm)

### Export Result Parameter for Further Analysis

#### Heart Rate

In [ ]:
bp.io.ecg.write_pandas_dict_excel(dict_phase, export_path.joinpath("hr_phase_export.xlsx"))
bp.io.ecg.write_pandas_dict_excel(dict_phase_norm, export_path.joinpath("hr_phase_export_normalized.xlsx"))

#### Heart Rate Variability

Concatenate the HRV result dict into one MultiIndex-DataFrame

In [ ]:
hrv_result = pd.concat(dict_hrv_result, names=["subject", "phase"])

hrv_result.to_csv(export_path.joinpath("hrv_params.csv"))
# alternatively:
# ecg.io.write_result_dict(dict_hrv_result, export_path.joinpath("hrv_params.csv"))